In [7]:
from IPython.display import Image

In [1]:
import torch

### tf32

In [8]:
Image(url='https://api.ibos.cn/v4/weapparticle/accesswximg?aid=86367&url=aHR0cHM6Ly9tbWJpei5xcGljLmNuL21tYml6X3BuZy9yY1Z1ZmlhVDAwVm5ZNTVQTnRnQ2dERVc0UzA5RVJKNERWSUtTRkZyRTZZOXVVUEFqQlJRTkpzc1owMjVjQUR2aFZvU2N5WVRJWnp3bFJnQnNKcmQ3bmcvNjQwP3d4X2ZtdD1wbmcmYW1w;from=appmsg', width=400)

- bf16 的问题范围大，但精度小；
- fp16 精度大，但范围小；

In [3]:
torch.randn(5).dtype

torch.float32

In [9]:
# Enable TF32 on CUDA
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

In [6]:
torch.randn(5, dtype='tf32').dtype

TypeError: randn() received an invalid combination of arguments - got (int, dtype=str), but expected one of:
 * (tuple of ints size, *, torch.Generator generator, tuple of names names, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (tuple of ints size, *, torch.Generator generator, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (tuple of ints size, *, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (tuple of ints size, *, tuple of names names, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
